# Install all of API keys for Groq

In [68]:
import os
groq_api_key = os.environ.get("GROQ_API_KEY")

# Install the source documents

In [69]:
#import json
#from langchain_community.document_loaders import PyPDFLoader

#source_folder = "/Users/presteddy56"
#with open(f"{source_folder}/resources/doc_name.json", 'r') as file:
#    data = json.load(file)
#loader ={}
#full_pages = {}
#for i in range(len(data['files'])):
#    country_name = data["countries"][i]
#    loader[country_name] = PyPDFLoader(f"{source_folder}/resources/{data["files"][i]}",extract_images=False)
#    full_pages[country_name] = loader[country_name].load_and_split()

In [70]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("/Users/presteddy56/resources/pdfs/")
docs = loader.load_and_split()
print(docs)

[Document(metadata={'source': '/Users/presteddy56/resources/pdfs/NICE_guideline.pdf', 'page': 0}, page_content='Depr ession in adul ts: \ntreatmen t and \nmanag emen t \nNICE guideline \nPublished: 29 June 2022 \nwww .nice.or g.uk/guidance/ng222 \n© NICE 202 4. All right s reserved. Subject t o Notice of right s (https://www .nice.or g.uk/t erms-and-\nconditions#notice-of -right s).'), Document(metadata={'source': '/Users/presteddy56/resources/pdfs/NICE_guideline.pdf', 'page': 1}, page_content='Your r esponsi bility \nThe r ecommendations in t his guideline r epresent t he view of NICE, arriv ed at aft er car eful \nconsideration of t he evidence a vailable. When e xercising t heir judgement, pr ofessionals \nand practitioners ar e expect ed to tak e this guideline fully int o account, alongside t he \nindividual needs, pr eferences and v alues of t heir patient s or t he people using t heir ser vice. \nIt is not mandat ory to apply t he recommendations, and t he guideline does not o v

In [71]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.1-70b-versatile", api_key=groq_api_key)

In [72]:
from langchain_huggingface import HuggingFaceEmbeddings
# Initialize the embedding model
embed_model = HuggingFaceEmbeddings(model_name = "mixedbread-ai/mxbai-embed-large-v1")

In [73]:
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(
    docs,
    embedding = embed_model,
    persist_directory = f"{source_folder}/resources",
    )

In [74]:
query = 'what is the best treatment for depression'
docs = vectorstore.similarity_search(query)
print(docs[0].page_content)

APA | Guideline for the Treatment of Depression  11For adult patients with depression, there is insufficient evidence to 
recommend for or against clinicians offering: 
• CBASP (cognitive behavioral analysis system of psychotherapy)
• Brief problem-solving therapy (10 or fewer sessions) vs. treatment 
as usual.Insufficient evidence for  
a recommendationBased on the literature reviewed that 
met the IOM or AMSTAR require-
ments the evidence was insufficient 
to be able to recommend for or 
against the listed interventions or 
treatment comparisons. 
Decision should be based on shared 
decision-making with the patient.
Complementary and Alternative Treatments18
For adults with depression for whom psychotherapy or pharmacothera-
py is either ineffective or unacceptable the panel suggests the following 
options:
• Exercise Monotherapy19
• St. John’s Wort Monotherapy20 
If neither is acceptable or available, the panel suggests consideration of 
• Bright light therapy21
• Yoga22
• If consid

In [75]:
retriever =vectorstore.as_retriever()

In [76]:
from langchain_core.prompts import PromptTemplate

template = (""" You are a medical doctor and advisor who precisely answers questions related to standard treatment in a diagnosis.
                Use the provided guidelines to answer the questions.
                If you don't know the answer, say so. Do not discuss the context in your response; just provide the answer directly. 
                Then, provide a reference in the given guidelines to review the details. 
                You should refer to the given guidelines as much as possible. 
                If you add something outside the given guideline, you should mention that references are from outside of the given guidelines.

                Guidelines:{guidelines}
                
                Question:{question}
                
                Answer:""")
rag_prompt = PromptTemplate.from_template(template)

In [77]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"guidelines": retriever, "question":RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [78]:
from IPython.display import display, Markdown
response = rag_chain.invoke("What is the best treatment for depression currently")
Markdown(response)

The guideline suggests that the best treatments for depression are psychotherapy or pharmacotherapy (antidepressant medication). However, for adults with depression for whom these options are either ineffective or unacceptable, the panel suggests considering the following complementary and alternative treatments: 

- Exercise Monotherapy 
- St. John’s Wort Monotherapy 
- Bright light therapy 
- Yoga 
- Acupuncture as an adjunct to antidepressant medication.

 Reference: APA Guideline for the Treatment of Depression, page 14.

In [79]:
import gradio as gr

def rag_memory_stream(text):
    partial_text =""
    for new_text in rag_chain.stream(text):
        partial_text += new_text
        # Yield an empty string to clearnup the message textbox and the updated conversation history
        yield partial_text
title = "Real-time AI App with Groq API and LangChain"
description = """<center> <img src="" alt = "logo" width="550"/></center>"""

demo = gr.Interface(
    title = title,
    description = description,
    fn = rag_memory_stream,
    inputs = "text",
    outputs= "text",
    live = False,
    allow_flagging="never",
)
demo.launch()

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
